# Neural Network - Training on Binary Lists

This notebook shows how to set up a training scenario using a neural network.
It will be used to feedforward a list of bits and convert it to a number.

## Code Implementation

### Libraries and Helper Functions

First, we need to import `NeuralNetwork`.

In [239]:
from datetime import datetime

import numpy as np

from neural_network.neural_network import NeuralNetwork


We can define some helper functions to analyse the results from the training.

In [240]:
def generate_time_msg() -> str:
    """
    Get message prefix with current datetime.
    """
    time_msg = f"[{datetime.now().strftime('%d-%m-%G | %H:%M:%S')}]"
    return time_msg


def print_system_msg(msg: str) -> None:
    """
    Print a message to the terminal.

    Parameters:
        msg (str): Message to print
    """
    print(f"{generate_time_msg()} {msg}")


def print_flushed_msg(msg: str) -> None:
    """
    Print a flushed message to the terminal.

    Parameters:
        msg (str): Message to print
    """
    print(f"\r{generate_time_msg()} {msg}", flush=True, end="")


def calculate_rms(errors: list[float]) -> float:
    """
    Calculate RMS from errors.

    Parameters:
        errors (list[float]): Errors from neural network training

    Returns:
        rms (float): RMS from errors
    """
    squared = np.square(errors)
    mean = np.average(squared)
    rms = np.sqrt(mean)
    return rms


def map_val(x: float, in_min: float, in_max: float, out_min: float, out_max: float) -> float:
    """
    Map a value from an input range to an output range.

    Parameters:
        x (float): Number to map to new range
        in_min (float): Lower bound of original range
        in_max (float): Upper bound of original range
        out_min (float): Lower bound of new range
        out_max (float): Upper bound of new range

    Returns:
        y (float): Number mapped to new range
    """
    y = (x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min
    return y


### Creating Methods to Generate Training Data

We will be using 8-bit numbers to train the neural network.
We can use the following bit map to convert numbers between integers and byte lists.

In [241]:
NUM_BITS = 8
BIT_MAP = np.array([2 ** (NUM_BITS - (i + 1)) for i in range(NUM_BITS)])
print_system_msg(f"Bit map: {BIT_MAP}")


def num_to_byte_list(num: int) -> list[int]:
    """
    Convert a number to a list of bits.

    Parameters:
        num (int): Number to convert

    Returns:
        byte_list (list[int]): Number represented as list of bits
    """
    _num_bin = bin(num)
    _num_bytes = _num_bin[2:]
    _padding = [0] * (NUM_BITS - len(_num_bytes))
    byte_list = _padding + [int(b) for b in _num_bytes]
    return byte_list


[24-04-2024 | 23:46:55] Bit map: [128  64  32  16   8   4   2   1]


With 8 bits, we can generate numbers between 0-255.
The neural network outputs numbers between 0-1 and therefore we need to map the values accordingly.

In [242]:
IN_LIMS = [0, 255]
OUT_LIMS = [0, 1]


def training_data_from_num(num: int) -> tuple[list[int], list[float]]:
    """
    Generate byte list and mapped number from a number to use in training.

    Parameters:
        num (int): Number to use for training data

    Returns:
        training_data (tuple[list[int], list[float]]): Input and expected output
    """
    _byte_list = num_to_byte_list(num)
    _mapped_num = map_val(num, IN_LIMS[0], IN_LIMS[1], OUT_LIMS[0], OUT_LIMS[1])
    training_data = (_byte_list, [_mapped_num])
    return training_data


### Creating the Neural Network

The number of inputs for the neural network is `NUM_BITS`.
The number of outputs is 1 as we will be converting a float to a number in our original range.


In [243]:
hidden_layer_sizes = [7, 5, 3]

nn = NeuralNetwork(NUM_BITS, 1, hidden_layer_sizes)


### Running the Algorithm

To train the neural network, we will select a random number and train the neural network with the corresponding byte list and expected output.

In [244]:
num_iters = 30000
print_system_msg(f"Beginning training with {num_iters} data points...")
for i in range(num_iters):
    random_num = np.random.randint(low=0, high=(IN_LIMS[1] + 1))
    training_input, expected_output = training_data_from_num(random_num)
    errors = nn.train(training_input, expected_output)
    rms = calculate_rms(errors)
    print_flushed_msg(f"{i+1} / {num_iters} -> RMS: {rms:.4f}")
print_flushed_msg(f"{num_iters} / {num_iters} -> Training complete! Final error: {rms:.4f}")


[24-04-2024 | 23:46:55] Beginning training with 30000 data points...
[24-04-2024 | 23:47:12] 30000 / 30000 -> Training complete! Final error: 0.0097

Now, we can test the neural network against some inputs and expected outputs to check its accuracy.

In [245]:
num_tests = 10
errors = []
print_system_msg(f"Beginning testing with {num_tests} numbers...")
for _ in range(num_tests):
    random_num = np.random.randint(low=0, high=(IN_LIMS[1] + 1))
    training_input, expected_output = training_data_from_num(random_num)
    output = nn.feedforward(training_input)[0]
    mapped_output = map_val(output, OUT_LIMS[0], OUT_LIMS[1], IN_LIMS[0], IN_LIMS[1])
    error = random_num - mapped_output
    errors.append(error)
    print_system_msg(f"Expected: {random_num} \t| Output: {mapped_output:.2f} \t| Error: {error:.2f}")

avg_error = np.average(errors)
print_system_msg(f"Testing complete! Average error: {avg_error:.2f}")


[24-04-2024 | 23:47:12] Beginning testing with 10 numbers...
[24-04-2024 | 23:47:12] Expected: 170 	| Output: 174.21 	| Error: -4.21
[24-04-2024 | 23:47:12] Expected: 247 	| Output: 239.53 	| Error: 7.47
[24-04-2024 | 23:47:12] Expected: 179 	| Output: 178.48 	| Error: 0.52
[24-04-2024 | 23:47:12] Expected: 38 	| Output: 37.06 	| Error: 0.94
[24-04-2024 | 23:47:12] Expected: 8 	| Output: 17.21 	| Error: -9.21
[24-04-2024 | 23:47:12] Expected: 181 	| Output: 180.25 	| Error: 0.75
[24-04-2024 | 23:47:12] Expected: 78 	| Output: 77.89 	| Error: 0.11
[24-04-2024 | 23:47:12] Expected: 48 	| Output: 55.67 	| Error: -7.67
[24-04-2024 | 23:47:12] Expected: 71 	| Output: 69.87 	| Error: 1.13
[24-04-2024 | 23:47:12] Expected: 215 	| Output: 215.05 	| Error: -0.05
[24-04-2024 | 23:47:12] Testing complete! Average error: -1.02
